In [28]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd
import numpy as np
import re

import json
import geopandas as gpd

In [30]:
### 통계청 행정구역 분류

df_code = pd.read_excel(f'{rawdata_path}/region_code/한국행정구역분류_2023.4.1.기준.xlsx', sheet_name='2-2. 연계표_행정동 및 법정동(기준시점)', header=1)
df_code = df_code[df_code['시도'] == '서울특별시']
df_code

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11.0,1100000000,19880423,1100000000,NaN,Seoul,NaN
1,서울특별시,종로구,종로구,종로구,종로구,11010.0,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
2,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720.0,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
3,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720.0,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,궁정동,11010720.0,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
762,서울특별시,강동구,성내2동,성내제2동,성내동,11250660.0,1174065000,19880423,1174010800,분할연계,Seongnae 2(i)-dong,NaN
763,서울특별시,강동구,성내3동,성내제3동,성내동,11250670.0,1174066000,19880423,1174010800,분할연계,Seongnae 3(sam)-dong,NaN
764,서울특별시,강동구,길동,길동,길동,11250740.0,1174068500,20080707,1174010500,NaN,Gil-dong,NaN
765,서울특별시,강동구,둔촌1동,둔촌제1동,둔촌동,11250700.0,1174069000,19880423,1174010600,분할연계,Dunchon 1(il)-dong,NaN


In [31]:
df_code = df_code[df_code['행정구역분류'] > 100000]
df_code.drop(columns=['시도', '행정동\n(행정기관명)', '행정기관 생성일', '법정동 관할구역\n분할여부', '행정동 영문명칭', '비고'], inplace= True)
df_code.reset_index(drop = True, inplace= True)
df_code.head()

<ipython-input-31-d54be3e9b6f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code.drop(columns=['시도', '행정동\n(행정기관명)', '행정기관 생성일', '법정동 관할구역\n분할여부', '행정동 영문명칭', '비고'], inplace= True)


,시군구,행정구역명,법정동,행정구역분류,행정기관코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500


In [32]:
df_code.columns = ['시군구', '행정동', '법정동', 'ADM_CD', '행정동코드', '법정동코드']
df_code.head()

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500


In [33]:
# 행안부 통계코드 분류

hang = pd.read_excel(f'{rawdata_path}/region_code/행안부_동코드 정리.xlsx')
hang.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [34]:
hang = hang[hang['시도명'] == '서울특별시']
hang = hang[hang['읍면동명'].notna()]
hang.head(8)

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN
5,1111051500,서울특별시,종로구,청운효자동,1111010400,효자동,20081101,NaN
6,1111051500,서울특별시,종로구,청운효자동,1111010500,창성동,20081101,NaN
7,1111051500,서울특별시,종로구,청운효자동,1111010800,통인동,20081101,NaN
8,1111051500,서울특별시,종로구,청운효자동,1111010900,누상동,20081101,NaN
9,1111051500,서울특별시,종로구,청운효자동,1111011000,누하동,20081101,NaN


In [35]:
hang.drop(columns=['시도명', '생성일자', '말소일자'], inplace = True)
hang.columns = ['행정동코드', '시군구', '행정동', '법정동코드', '법정동']
hang.reset_index(drop=True, inplace=True)
hang

,행정동코드,시군구,행정동,법정동코드,법정동
0,1111051500,종로구,청운효자동,1111010100,청운동
1,1111051500,종로구,청운효자동,1111010200,신교동
2,1111051500,종로구,청운효자동,1111010300,궁정동
3,1111051500,종로구,청운효자동,1111010400,효자동
4,1111051500,종로구,청운효자동,1111010500,창성동
...,...,...,...,...,...
740,1174065000,강동구,성내제2동,1174010800,성내동
741,1174066000,강동구,성내제3동,1174010800,성내동
742,1174068500,강동구,길동,1174010500,길동
743,1174069000,강동구,둔촌제1동,1174010600,둔촌동


In [36]:
df_code

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500
...,...,...,...,...,...,...
736,강동구,성내2동,성내동,11250660.0,1174065000,1174010800
737,강동구,성내3동,성내동,11250670.0,1174066000,1174010800
738,강동구,길동,길동,11250740.0,1174068500,1174010500
739,강동구,둔촌1동,둔촌동,11250700.0,1174069000,1174010600


In [37]:
hang[hang['행정동'] == '청운효자동']

,행정동코드,시군구,행정동,법정동코드,법정동
0,1111051500,종로구,청운효자동,1111010100,청운동
1,1111051500,종로구,청운효자동,1111010200,신교동
2,1111051500,종로구,청운효자동,1111010300,궁정동
3,1111051500,종로구,청운효자동,1111010400,효자동
4,1111051500,종로구,청운효자동,1111010500,창성동
5,1111051500,종로구,청운효자동,1111010800,통인동
6,1111051500,종로구,청운효자동,1111010900,누상동
7,1111051500,종로구,청운효자동,1111011000,누하동
8,1111051500,종로구,청운효자동,1111011100,옥인동
9,1111051500,종로구,청운효자동,1111011900,세종로


In [38]:
df_code[df_code['행정동'] == '청운효자동']

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500
5,종로구,청운효자동,통인동,11010720.0,1111051500,1111010800
6,종로구,청운효자동,누상동,11010720.0,1111051500,1111010900
7,종로구,청운효자동,누하동,11010720.0,1111051500,1111011000
8,종로구,청운효자동,옥인동,11010720.0,1111051500,1111011100
9,종로구,청운효자동,세종로,11010720.0,1111051500,1111011900


In [39]:
temp = df_code.merge(hang,
                     how = 'outer',
                     on= ['시군구', '행정동코드', '법정동코드'],
                     suffixes=['1', '2'])

In [40]:
temp

,시군구,행정동1,법정동1,ADM_CD,행정동코드,법정동코드,행정동2,법정동2
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100,청운효자동,청운동
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200,청운효자동,신교동
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300,청운효자동,궁정동
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400,청운효자동,효자동
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500,청운효자동,창성동
...,...,...,...,...,...,...,...,...
740,강동구,둔촌2동,둔촌동,11250710.0,1174070000,1174010600,둔촌제2동,둔촌동
741,노원구,NaN,NaN,NaN,1135062400,1135010500,중계4동,상계동
742,서대문구,NaN,NaN,NaN,1141055500,1141011200,북아현동,대현동
743,강서구,NaN,NaN,NaN,1150060300,1150010200,가양제1동,등촌동


In [41]:
temp[temp.isna().any(axis=1)]

,시군구,행정동1,법정동1,ADM_CD,행정동코드,법정동코드,행정동2,법정동2
741,노원구,NaN,NaN,NaN,1135062400,1135010500,중계4동,상계동
742,서대문구,NaN,NaN,NaN,1141055500,1141011200,북아현동,대현동
743,강서구,NaN,NaN,NaN,1150060300,1150010200,가양제1동,등촌동
744,강서구,NaN,NaN,NaN,1150064100,1150010500,방화제3동,마곡동


In [42]:
temp.dropna(inplace=True)
region_code = temp.copy()

In [43]:
region_code['행정동1'] = region_code['행정동1'].apply(lambda x: re.sub(r'\.', '', x))
region_code['행정동1'] = region_code['행정동1'].apply(lambda x: re.sub(r'\·', '', x))

region_code['행정동2'] = region_code['행정동2'].apply(lambda x: re.sub(r'\.', '', x))
region_code['행정동2'] = region_code['행정동2'].apply(lambda x: re.sub(r'\·', '', x))

region_code['법정동1'] = region_code['법정동1'].apply(lambda x: re.sub(r'\.', '', x))
region_code['법정동1'] = region_code['법정동1'].apply(lambda x: re.sub(r'\·', '', x))

region_code['법정동2'] = region_code['법정동2'].apply(lambda x: re.sub(r'\.', '', x))
region_code['법정동2'] = region_code['법정동2'].apply(lambda x: re.sub(r'\·', '', x))




In [44]:
region_code = region_code[['ADM_CD', '행정동코드', '법정동코드', '시군구', '행정동1', '행정동2', '법정동1', '법정동2']]
region_code['key1'] = region_code['시군구'] + ' ' + region_code['행정동1']
region_code['key2'] = region_code['시군구'] + ' ' + region_code['행정동2']
region_code

<ipython-input-44-15bfb54dc720>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_code['key1'] = region_code['시군구'] + ' ' + region_code['행정동1']
<ipython-input-44-15bfb54dc720>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_code['key2'] = region_code['시군구'] + ' ' + region_code['행정동2']


,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동
...,...,...,...,...,...,...,...,...,...,...
736,11250660.0,1174065000,1174010800,강동구,성내2동,성내제2동,성내동,성내동,강동구 성내2동,강동구 성내제2동
737,11250670.0,1174066000,1174010800,강동구,성내3동,성내제3동,성내동,성내동,강동구 성내3동,강동구 성내제3동
738,11250740.0,1174068500,1174010500,강동구,길동,길동,길동,길동,강동구 길동,강동구 길동
739,11250700.0,1174069000,1174010600,강동구,둔촌1동,둔촌제1동,둔촌동,둔촌동,강동구 둔촌1동,강동구 둔촌제1동


In [45]:
region_code[region_code['행정동1'] == '청운효자동']

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동
5,11010720.0,1111051500,1111010800,종로구,청운효자동,청운효자동,통인동,통인동,종로구 청운효자동,종로구 청운효자동
6,11010720.0,1111051500,1111010900,종로구,청운효자동,청운효자동,누상동,누상동,종로구 청운효자동,종로구 청운효자동
7,11010720.0,1111051500,1111011000,종로구,청운효자동,청운효자동,누하동,누하동,종로구 청운효자동,종로구 청운효자동
8,11010720.0,1111051500,1111011100,종로구,청운효자동,청운효자동,옥인동,옥인동,종로구 청운효자동,종로구 청운효자동
9,11010720.0,1111051500,1111011900,종로구,청운효자동,청운효자동,세종로,세종로,종로구 청운효자동,종로구 청운효자동


In [46]:
region_code[region_code['법정동1'] != region_code['법정동2']]

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2


In [47]:
region_code[region_code['행정동1'] != region_code['행정동2']]

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
88,11010670.0,1111067000,1111017400,종로구,창신1동,창신제1동,창신동,창신동,종로구 창신1동,종로구 창신제1동
89,11010680.0,1111068000,1111017400,종로구,창신2동,창신제2동,창신동,창신동,종로구 창신2동,종로구 창신제2동
90,11010690.0,1111069000,1111017400,종로구,창신3동,창신제3동,창신동,창신동,종로구 창신3동,종로구 창신제3동
91,11010700.0,1111070000,1111017500,종로구,숭인1동,숭인제1동,숭인동,숭인동,종로구 숭인1동,종로구 숭인제1동
92,11010710.0,1111071000,1111017500,종로구,숭인2동,숭인제2동,숭인동,숭인동,종로구 숭인2동,종로구 숭인제2동
...,...,...,...,...,...,...,...,...,...,...
735,11250650.0,1174064000,1174010800,강동구,성내1동,성내제1동,성내동,성내동,강동구 성내1동,강동구 성내제1동
736,11250660.0,1174065000,1174010800,강동구,성내2동,성내제2동,성내동,성내동,강동구 성내2동,강동구 성내제2동
737,11250670.0,1174066000,1174010800,강동구,성내3동,성내제3동,성내동,성내동,강동구 성내3동,강동구 성내제3동
739,11250700.0,1174069000,1174010600,강동구,둔촌1동,둔촌제1동,둔촌동,둔촌동,강동구 둔촌1동,강동구 둔촌제1동


In [53]:
region_code[region_code['행정동2'] == '신사동']

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
630,11210680.0,1162068500,1162010200,관악구,신사동,신사동,신림동,신림동,관악구 신사동,관악구 신사동
661,11230510.0,1168051000,1168010700,강남구,신사동,신사동,신사동,신사동,강남구 신사동,강남구 신사동
662,11230510.0,1168051000,1168011000,강남구,신사동,신사동,압구정동,압구정동,강남구 신사동,강남구 신사동


In [54]:
region_code[region_code['시군구'] == '은평구']

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
392,11120510.0,1138051000,1138010200,은평구,녹번동,녹번동,녹번동,녹번동,은평구 녹번동,은평구 녹번동
393,11120520.0,1138052000,1138010300,은평구,불광1동,불광제1동,불광동,불광동,은평구 불광1동,은평구 불광제1동
394,11120720.0,1138053000,1138010300,은평구,불광2동,불광제2동,불광동,불광동,은평구 불광2동,은평구 불광제2동
395,11120550.0,1138055100,1138010400,은평구,갈현1동,갈현제1동,갈현동,갈현동,은평구 갈현1동,은평구 갈현제1동
396,11120560.0,1138055200,1138010400,은평구,갈현2동,갈현제2동,갈현동,갈현동,은평구 갈현2동,은평구 갈현제2동
397,11120570.0,1138056000,1138010500,은평구,구산동,구산동,구산동,구산동,은평구 구산동,은평구 구산동
398,11120580.0,1138057000,1138010600,은평구,대조동,대조동,대조동,대조동,은평구 대조동,은평구 대조동
399,11120590.0,1138058000,1138010700,은평구,응암1동,응암제1동,응암동,응암동,은평구 응암1동,은평구 응암제1동
400,11120600.0,1138059000,1138010700,은평구,응암2동,응암제2동,응암동,응암동,은평구 응암2동,은평구 응암제2동
401,11120730.0,1138060000,1138010700,은평구,응암3동,응암제3동,응암동,응암동,은평구 응암3동,은평구 응암제3동


In [51]:
region_code.to_csv(f'{data_path}/region_code_all.csv', index=False)

In [52]:
region_code2 = region_code.drop(columns = ['법정동코드', '법정동1', '법정동2'])
region_code2.drop_duplicates(inplace = True)
region_code2.reset_index(drop = True, inplace = True)
region_code2

,ADM_CD,행정동코드,시군구,행정동1,행정동2,key1,key2
0,11010720.0,1111051500,종로구,청운효자동,청운효자동,종로구 청운효자동,종로구 청운효자동
1,11010530.0,1111053000,종로구,사직동,사직동,종로구 사직동,종로구 사직동
2,11010540.0,1111054000,종로구,삼청동,삼청동,종로구 삼청동,종로구 삼청동
3,11010550.0,1111055000,종로구,부암동,부암동,종로구 부암동,종로구 부암동
4,11010560.0,1111056000,종로구,평창동,평창동,종로구 평창동,종로구 평창동
...,...,...,...,...,...,...,...
421,11250660.0,1174065000,강동구,성내2동,성내제2동,강동구 성내2동,강동구 성내제2동
422,11250670.0,1174066000,강동구,성내3동,성내제3동,강동구 성내3동,강동구 성내제3동
423,11250740.0,1174068500,강동구,길동,길동,강동구 길동,강동구 길동
424,11250700.0,1174069000,강동구,둔촌1동,둔촌제1동,강동구 둔촌1동,강동구 둔촌제1동


In [ ]:
region_code2.to_csv(f'{data_path}/region_code.csv', index=False)

In [ ]:
dong_shp = gpd.read_file(f'{rawdata_path}/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')
dong_shp

,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1..."
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1..."
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1..."
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1..."
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1..."
...,...,...,...,...
421,20230701,가회동,11010600,"POLYGON ((954894.795 1954614.580, 954888.290 1..."
422,20230701,종로1·2·3·4가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1..."
423,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1..."
424,20230701,미아동,11090700,"POLYGON ((958301.054 1959603.612, 958292.609 1..."


In [ ]:
dong_shp['ADM_CD'] =  (dong_shp['ADM_CD']).astype(int)
# hang['ADM_CD'] =  hang['ADM_CD'].astype(int)

temp2 = dong_shp.merge(region_code2,
               how = 'left',
               on = 'ADM_CD')

In [ ]:
temp2

,BASE_DATE,ADM_NM,ADM_CD,geometry,행정동코드,시군구,행정동1,행정동2,key1,key2
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",1111053000,종로구,사직동,사직동,종로구 사직동,종로구 사직동
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",1111054000,종로구,삼청동,삼청동,종로구 삼청동,종로구 삼청동
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",1111055000,종로구,부암동,부암동,종로구 부암동,종로구 부암동
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",1111056000,종로구,평창동,평창동,종로구 평창동,종로구 평창동
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",1117068500,용산구,한남동,한남동,용산구 한남동,용산구 한남동
...,...,...,...,...,...,...,...,...,...,...
421,20230701,가회동,11010600,"POLYGON ((954894.795 1954614.580, 954888.290 1...",1111060000,종로구,가회동,가회동,종로구 가회동,종로구 가회동
422,20230701,종로1·2·3·4가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1...",1111061500,종로구,종로1234가동,종로1234가동,종로구 종로1234가동,종로구 종로1234가동
423,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1...",1165061000,서초구,방배2동,방배2동,서초구 방배2동,서초구 방배2동
424,20230701,미아동,11090700,"POLYGON ((958301.054 1959603.612, 958292.609 1...",1130553500,강북구,미아동,미아동,강북구 미아동,강북구 미아동


In [ ]:
temp2[temp2.isna().any(axis = 1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,행정동코드,시군구,행정동1,행정동2,key1,key2
